In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
'''import os''' #some of you might need this
import warnings
warnings.filterwarnings('ignore') # For better display; it's usually NOT a good idea to suppress warnings

In [ ]:
'''os.chdir('path_to_data')'''
eeglab_raw = mne.io.read_raw_eeglab('S1.set')
eeglab_epochs = mne.read_epochs_eeglab('S1_elist_bins_chop_be.set')

eeglab_epochs.event_id

In [ ]:
Evokeds_Dict = namedtuple('Evokeds_Dict', ['Evokeds_List','All_Bins'])
evokeds_dict = Evokeds_Dict([eeglab_epochs[v].average() for v in ['B1(221)','B2(222)','B3(221)','B4(222)']],
                            ['B1(221)','B2(222)','B3(221)','B4(222)'])

In [ ]:
evokeds, all_bins = evokeds_dict

trange = (-200, 1200)
first, t = evokeds[0].first, [i for i in range(trange[0], trange[1])]

n_bins = len(evokeds)
n_chs, n_times = evokeds[0].get_data().shape
data = np.array([evokeds[i].get_data(units='uV')
                 for i in range(len(evokeds))], dtype=object).reshape(n_bins, n_chs, n_times)[:,:,trange[0]-first:trange[1]-first]

ch_dict = {}
for i, x in enumerate(evokeds[0].ch_names):
    ch_dict[x] = i

chs_to_plot, bins_to_plot = ['FZ','CZ','PZ'], ['B1(221)','B2(222)']
linestyles, colors = ['solid','solid'], ['navy','deepskyblue']

savefig, filepath = False, None

In [ ]:
nrows, ncols = 3, 1
figsize = (12, 15)

figure, axes = plt.subplots(nrows, ncols, figsize=figsize, sharey=True)
for ax, ch in zip(axes.copy().flatten(), chs_to_plot):
    for i, x in enumerate(bins_to_plot):
        ax.plot(t, data[all_bins.index(x)][ch_dict[ch]],
                linestyle=linestyles[i], color=colors[i], label=x)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(ch)
    ax.set_xlabel('Time (ms)')
    ax.set_xlim(trange[0], trange[1])
    ax.set_xticks(list(range(-200,1400,200)))
    ax.set_ylim(-10, 10)
    ax.set_yticks(list(range(-10,10,5)))
    ax.set_ylabel('µV')
    ax.invert_yaxis()
    ax.yaxis.set_tick_params(labelbottom=True)
    hdl, lbl = ax.get_legend_handles_labels()
figure.legend(hdl, lbl, loc='upper left', bbox_to_anchor=[-0.001, 0.001])
figure.tight_layout()
if savefig == True: plt.savefig(filepath)
plt.show()

In [ ]:
chs_to_plot = evokeds[0].ch_names
pos_dict = {'HEO':1,'FP1':2,'FP2':4,'VEO':5,
                'F7':8,'F3':9,'FZ':10,'F4':11,'F8':12,
                'FT7':15,'FC3':16,'FCZ':17,'FC4':18,'FT8':19,
                'T7':22,'C3':23,'CZ':24,'C4':25,'T8':26,
                'M1':28,'TP7':29,'CP3':30,'CPZ':31,'CP4':32,'TP8':33,'M2':34,
                'P7':36,'P3':37,'PZ':38,'P4':39,'P8':40,
                'O1':44,'OZ':45,'O2':46}
keys, vals = list(pos_dict.keys()), list(pos_dict.values())

nrows, ncols = 7, 7
figsize = (35,25)
figure, axes = plt.subplots(nrows, ncols, figsize=figsize, sharey=True)
for (m,n), ax in np.ndenumerate(axes):
    if m*ncols+n == 45: ax.set_xlabel('Time (ms)', fontsize=22, labelpad=20.0)
    if m*ncols+n == 28: ax.set_ylabel('µV', fontsize=22, labelpad=20.0)
    try:
        if keys[vals.index(m*ncols+n)] not in chs_to_plot: ax.remove()
    except ValueError:
        ax.remove()
            
for ch in chs_to_plot:
    ax = plt.subplot(nrows, ncols, pos_dict[ch]+1)
    for i, x in enumerate(bins_to_plot):
        ax.plot(t, data[all_bins.index(x)][ch_dict[ch]], linestyle=linestyles[i], color=colors[i], label=x)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(ch, fontsize=25)
    ax.set_xlim(trange[0], trange[1])
    ax.set_xticks([-200,0,300,600,900,1200])
    ax.set_ylim(-20, 20)
    ax.set_yticks(list(range(-20,20,10)))
    ax.invert_yaxis()
    ax.yaxis.set_tick_params(labelbottom=True, labelsize=16)
    ax.xaxis.set_tick_params(labelbottom=True, labelsize=16)
    hdl, lbl = ax.get_legend_handles_labels()
figure.legend(hdl, lbl, loc='lower left', bbox_to_anchor=[-0.001, -0.001], fontsize=25)
figure.tight_layout()
if savefig == True: plt.savefig(filepath)
plt.show()